In [1]:
import json
import openai
import os
import re
from langchain_openai import ChatOpenAI
import transformers
from datasets import Dataset

In [9]:
file_path = 'C:\\Users\\PC\\Desktop\\DoYoung\\DS\\github\\bitamin_auto_readme_generator\\data\\object_detection\\output\\ocr_samples_txt\\netflix_text.txt'

with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

In [3]:
api_key_filepath = "C:\\Users\\PC\\Desktop\\DoYoung\\DS\\비타민NLP_240701\\text_summarization\\openai_api_key.json"
with open(api_key_filepath, 'r') as f:
    api_key = json.load(f)
api_key = api_key['OPENAI_API_KEY']

In [4]:
os.environ['OPENAI_API_KEY'] = api_key

In [5]:
llm_3 = ChatOpenAI(
    model="gpt-3.5-turbo",
    openai_api_key=os.environ['OPENAI_API_KEY']
)

In [6]:
llm_4 = ChatOpenAI(
    model="gpt-4o",
    openai_api_key=os.environ['OPENAI_API_KEY']
)

# Method3

### STEP1: 첫 5페이지에서 팀원, 제목, 목차 추출

In [7]:
end_marker = text.find('<p.6>')

if end_marker != -1:
    text5 = text[:end_marker]
else:
    text5 = text

In [8]:
def extract_info(text):
    prompt = f"""
    Extract the following information from the provided text:
    1. Name (excluding the team name, include all people listed)
    2. Title (include the entire title as it appears in the text)
    3. Main Topics (only main topics from the Table of Contents, excluding subtopics)

    The Main Topics should be extracted from the section that follows headers such as 'TABLE OF CONTENTS', '목차 소개', or any similar variation.
    Ensure that team member's name are not split into multiple lines. Name should be connected by commas if there are more than one.
    Ensure that title is not cut off and is extracted in their entirety.
    Ensure that main topics are not split into multiple lines. Name should be connected by commas if there are more than one.
    Exclude any subtopics or secondary information while extracting main topics.
    Stop at the next page marker '<p.'.

    Do not include any additional notes or explanations in the output.

    Text: {text}


    Format the extracted information as follows:
    <subject>title</subject>
    <team>team members</team>
    <index>main topics</index>

    """

    response = llm_4.invoke(prompt)
    extracted_info = response.content.strip()
    return extracted_info

extracted_info = extract_info(text5)
print(extracted_info)


<subject>Netflix Stock Price Prediction with News Topic & Sentiment</subject>
<team>송규헌, 권도영, 이태경, 김서윤, 한진솔</team>
<index>INTRODUCTION, DATA PREPROCESSING, MODELING, CONCLUSIONS AND LIMITATION</index>


In [10]:
main_topics_match = re.search(r'<index>(.*?)</index>', extracted_info, re.DOTALL)
if main_topics_match:
    main_topics = main_topics_match.group(1).strip()
    main_topics_list = [topic.strip() for topic in main_topics.split(',')]
else:
    main_topics_list = []

### STEP2: 전체 텍스트를 대주제 개수에 따라 구분

In [11]:
def remove_page(text):
    cleaned_text = re.sub(r'<p\.\d+>', '', text)
    return cleaned_text

In [12]:
cleaned_text = remove_page(text)
print(cleaned_text)


 2024 BITAmin 겨울 연합프로젝트 시계열1조
 Netflix Stock Price Prediction with News Topic & Sentiment
 시계열 1조 12기 송규헌 12기 권도영 12기이태경 13기 김서윤 13기 한진솔

 CONTENTS
 O1 INTRODUCTION
 04 CONCLUSIONS AND LIMIT ATION
 03 MODELING
 02 DATA PREPROCESSING

 01 INTRODU

 01 INTRODUCTION
 2024 BITAmin 겨울 연합프로젝트 시계열 1조
 11 Background of topic selection
 1 뉴스가 주가 변동에 미치는 영향 탐구 주가 예측에 뉴스를 활용할 수 있는지탐구해보고자 함 주가를 예측하는데 사용하는 데이터로 뉴스의 감성분석및 토픽 모델링 결과를 사용하고자 뉴스 기사에서는 주로 한 기업에 대해 보도하고 있어 예측 대상은 한 개의주식종목으로 정함 뉴스 감성분석/토픽 모델링 결과를 활용하여 주가를 예측하는 프로젝트는 많지 않아 직접 뉴스 데이터를 확용하고자 함

 01 INTRODUCTION
 2024 BITAmin 겨울 연합프로젝트 시계열 1조
 11 Background of topic selection
 2 장기적인 추세를 고려할 수 있는 주가 예측 모델 구현 기존의 주가 예측 프로젝트는 주로 모델의 예측 일수나 시권스 길이를 1~5로 설정함 짧은 시간 동안의 주가 변동을 예측한 프로젝트와 달리 예측 일수와 시권스 길이를 늘려 단기적인 변동뿐만 아니라 장기적인 추세를 고려한 예측 결과를 얻고자 함 기존 프로젝트에서 주가 예측 그래프가 실제주가 그래프를 단순 평행 이동한 형태로 나타나는 문제를 해결하고자함

 O1 INTRODUCTION
 2024 BITAmin 겨울 연합프로젝트 시계열 1조
 12 Brief Project Introduction
 [목표] 뉴스 데이터를 활용한 주가 예측 모델의 최적화 및 효율적인 파라미터 선정
 [주요 내용] 장기적인 추세 

In [13]:
def divide_text(extracted_info, main_topics_list, text):
    prompt = f"""
    The provided text contains specific sections that need to be excluded before dividing it according to the main topics.
    Exclude the following sections from the text:
    {extracted_info}

    After exclusion, divide the provided text according to the main topics.

    The text should be divided into exactly {len(main_topics_list)} sections, separated by "===================================".

    Use the main topics provided below as the indices for division:
    {main_topics_list}

    The text should remain unmodified.
    The text should only be partitioned according to the number of main topics.

    Text: {text}
    """

    response = llm_4.invoke(prompt)
    divided_text = response.content.strip()
    return divided_text


divided_text = divide_text(extracted_info, main_topics_list, cleaned_text)
print(divided_text)


INTRODUCTION
 01 INTRODUCTION
 2024 BITAmin 겨울 연합프로젝트 시계열 1조
 11 Background of topic selection
 1 뉴스가 주가 변동에 미치는 영향 탐구 주가 예측에 뉴스를 활용할 수 있는지탐구해보고자 함 주가를 예측하는데 사용하는 데이터로 뉴스의 감성분석및 토픽 모델링 결과를 사용하고자 뉴스 기사에서는 주로 한 기업에 대해 보도하고 있어 예측 대상은 한 개의주식종목으로 정함 뉴스 감성분석/토픽 모델링 결과를 활용하여 주가를 예측하는 프로젝트는 많지 않아 직접 뉴스 데이터를 확용하고자 함

 01 INTRODUCTION
 2024 BITAmin 겨울 연합프로젝트 시계열 1조
 11 Background of topic selection
 2 장기적인 추세를 고려할 수 있는 주가 예측 모델 구현 기존의 주가 예측 프로젝트는 주로 모델의 예측 일수나 시권스 길이를 1~5로 설정함 짧은 시간 동안의 주가 변동을 예측한 프로젝트와 달리 예측 일수와 시권스 길이를 늘려 단기적인 변동뿐만 아니라 장기적인 추세를 고려한 예측 결과를 얻고자 함 기존 프로젝트에서 주가 예측 그래프가 실제주가 그래프를 단순 평행 이동한 형태로 나타나는 문제를 해결하고자함

 O1 INTRODUCTION
 2024 BITAmin 겨울 연합프로젝트 시계열 1조
 12 Brief Project Introduction
 [목표] 뉴스 데이터를 활용한 주가 예측 모델의 최적화 및 효율적인 파라미터 선정
 [주요 내용] 장기적인 추세 예측에 효과적인 모델 판별: LSTM VS GRU Vs Transformer 실험을 통한 모델 파라미터 튜닝 뉴스 감성분석과 토픽모델링의 주가 예측 유용성 검토 각실험 결과 평가에 필요한 지표 선정
 [개발 환경] Colab

 01
 2024 BTAmin 겨울 연합프로젝트 시계열 1조
 13 Data collection
 1 주가 데이터 FinanceDataReader 라이브러리를 활용하여 NETFLIX 기업의 20

### 요약

In [14]:
def summarize(text):
    prompt = f"""
    The provided text is divided into several partitions separated by "===================================".
    Summarize each partition separately. 
    Do not change the language of the original input text. 
    Summarize in the same language as the input text.
    The purpose of the summaries is to create a comprehensive README for a GitHub project.

    Ensure that each summary accurately captures the key points and essential information of its respective partition.
    Summarize in a way that is concise and informative, omitting repetitive mentions of the main topic after the first mention in each partition.
    Focus on the key details and results relevant to the project.

    Do not remove or modify the "===================================" separators.

    Here is an example of a good summary:

    ===================================
    프로젝트 소개
    - 프로젝트 배경: 정제된 '대량의 데이터'를 사용하여 모델 선택 범위를 넓히고자 함.
    - 프로젝트 목표: 사용자 데이터, 웹툰 데이터를 이용해 개인화된 추천 시스템 개발. 사용자와 아이템의 interaction 데이터에 맞는 알고리즘 탐색 및 적용.
    ===================================
    데이터 수집 및 전처리
    - 데이터 소스 설명: 다양한 데이터 소스 사용. 데이터 전처리 과정 설명.
    - 데이터 처리: 피봇 테이블 형식의 데이터 생성, 데이터 정제 및 전처리.
    ===================================

    Text: {text}
    """

    response = llm_4.invoke(prompt)
    result = response.content.strip()
    return result

In [15]:
summarized_text = summarize(divided_text)
print(summarized_text)

INTRODUCTION
- 프로젝트 배경: 뉴스가 주가 변동에 미치는 영향을 탐구하고, 뉴스 감성분석 및 토픽 모델링을 사용하여 주가를 예측. 장기적인 추세를 고려한 주가 예측 모델 구현.
- 프로젝트 목표: 뉴스 데이터를 활용한 주가 예측 모델의 최적화 및 파라미터 선정. LSTM, GRU, Transformer 모델을 비교하여 최적의 모델 판별.
- 데이터 수집: FinanceDataReader 라이브러리를 통해 2018년 1월 2일부터 2023년 12월 29일까지 NETFLIX, 핀터레스트, 메타플랫폼스, 스포티파이의 주가 데이터 수집. Stock News API를 통해 같은 기간의 NETFLIX 뉴스 데이터 수집.

DATA PREPROCESSING
- 파생 변수 생성: 변화율, 이동 평균 등 다양한 파생 변수 생성. 2018년 데이터를 추가로 수집하여 변수 생성에 활용.
- 기술 분석 지표 추가: Bollinger Bands, Keltner Channel, Moving Average 등 37개의 기술 분석 지표 추가.
- 유사 주가 데이터 추가: 핀터레스트, 메타플랫폼스, 스포티파이의 종가를 추가하여 분석.
- 다중공선성 제거: 주가 변화 관련 기본 지표 유지, 상관관계 높은 보조 지표 제거.
- 뉴스 주제 및 감성 파생 변수 생성: 뉴스 토픽과 감성을 레이블 인코딩 및 원핫 인코딩하여 과거 뉴스 데이터의 영향을 고려.
- 최종 데이터셋: 주식 데이터 셋(1257*38), 전체 데이터 셋(1257*59).

MODELING
- 시계열 모델: 시간에 따른 데이터 패턴을 분석하여 미래 값을 예측. Sequence Length 60, Predict Size 10 설정.
- 모델 정의: LSTM, GRU, Transformer 모델 사용. 각 모델의 특징과 예측 방식 설명.
- 모델 비교: CLOSE vs 1d_ROC, 뉴스 포함 vs 미포함, LSTM vs GRU vs Transformer 모델 비교.

CONCLUSIONS AND LIMITATIONS
-

In [16]:
def tag_text(text):
    prompt = f"""
    Tag each part of the text according to the following rules:
    - The first line before any "-" should be tagged as <main>.
    - Each line starting with "-" should have the part before ":" tagged as <sub> and the part after ":" tagged as <content>.
    - Remove "-", "=", and ":" characters from the text.
    - Ensure the tags are correctly closed and formatted.
    Ensure that the output does not contain any <xml> tags.

    Here is an example of how the tagged text should look:

    <main>대한민국의 계절</main>
    <sub>여름</sub> <content>요즘 한국의 여름은 매우 덥고 습한 편이다.</content>
    <sub>겨울</sub> <content>12월부터 온도가 급격히 떨어지며 건조해진다.</content>

    <main>대한민국의 휴일</main>
    <sub>추석</sub> <content>추석은 비교적 휴일 기간이 긴 편이다.</content>
    <sub>설날</sub> <content>설날에는 떡국을 먹으며, 가족들과 인사를 나눈다.</content>

    Text:
    {text}
    """

    response = llm_4.invoke(prompt)
    result = response.content.strip()
    return result


In [17]:
tage_result = tag_text(summarized_text)
print(tage_result)

<main>INTRODUCTION</main>
<sub>프로젝트 배경</sub> <content>뉴스가 주가 변동에 미치는 영향을 탐구하고, 뉴스 감성분석 및 토픽 모델링을 사용하여 주가를 예측. 장기적인 추세를 고려한 주가 예측 모델 구현.</content>
<sub>프로젝트 목표</sub> <content>뉴스 데이터를 활용한 주가 예측 모델의 최적화 및 파라미터 선정. LSTM, GRU, Transformer 모델을 비교하여 최적의 모델 판별.</content>
<sub>데이터 수집</sub> <content>FinanceDataReader 라이브러리를 통해 2018년 1월 2일부터 2023년 12월 29일까지 NETFLIX, 핀터레스트, 메타플랫폼스, 스포티파이의 주가 데이터 수집. Stock News API를 통해 같은 기간의 NETFLIX 뉴스 데이터 수집.</content>

<main>DATA PREPROCESSING</main>
<sub>파생 변수 생성</sub> <content>변화율, 이동 평균 등 다양한 파생 변수 생성. 2018년 데이터를 추가로 수집하여 변수 생성에 활용.</content>
<sub>기술 분석 지표 추가</sub> <content>Bollinger Bands, Keltner Channel, Moving Average 등 37개의 기술 분석 지표 추가.</content>
<sub>유사 주가 데이터 추가</sub> <content>핀터레스트, 메타플랫폼스, 스포티파이의 종가를 추가하여 분석.</content>
<sub>다중공선성 제거</sub> <content>주가 변화 관련 기본 지표 유지, 상관관계 높은 보조 지표 제거.</content>
<sub>뉴스 주제 및 감성 파생 변수 생성</sub> <content>뉴스 토픽과 감성을 레이블 인코딩 및 원핫 인코딩하여 과거 뉴스 데이터의 영향을 고려.</content>
<sub>최종 데이터셋</sub> <content>주식 데이터 셋(1257*38), 전체 데이터 셋(1257